See [guide](https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5)

In [1]:
import pandas as pd
import torch
import wandb
import src.Baseline

from torch.utils.data import DataLoader
from torch.utils.data import random_split
from importlib import reload
from src.Baseline import SoundDS, AudioClassifier, training, inference

# reload Baseline modules
reload(src.Baseline)

<module 'src.Baseline' from 'C:\\code\\dspro2\\src\\Baseline.py'>

In [2]:
# only use aldfly and bkcchi for example purposes
df = pd.read_csv('input/birdsong-recognition/train.csv')

df = df[df['ebird_code'].isin(['aldfly', 'bkcchi'])]
df = df.reset_index(drop=True)
df.head()

,rating,playback_used,ebird_code,channels,date,pitch,duration,filename,speed,species,...,xc_id,url,country,author,primary_label,longitude,length,time,recordist,license
0,3.5,no,aldfly,1 (mono),2013-05-25,Not specified,25,XC134874.mp3,Not specified,Alder Flycatcher,...,134874,https://www.xeno-canto.org/134874,United States,Jonathon Jongsma,Empidonax alnorum_Alder Flycatcher,-92.962,Not specified,8:00,Jonathon Jongsma,Creative Commons Attribution-ShareAlike 3.0
1,4.0,no,aldfly,2 (stereo),2013-05-27,both,36,XC135454.mp3,both,Alder Flycatcher,...,135454,https://www.xeno-canto.org/135454,United States,Mike Nelson,Empidonax alnorum_Alder Flycatcher,-82.1106,0-3(s),08:30,Mike Nelson,Creative Commons Attribution-NonCommercial-Sha...
2,4.0,no,aldfly,2 (stereo),2013-05-27,both,39,XC135455.mp3,both,Alder Flycatcher,...,135455,https://www.xeno-canto.org/135455,United States,Mike Nelson,Empidonax alnorum_Alder Flycatcher,-82.1106,0-3(s),08:30,Mike Nelson,Creative Commons Attribution-NonCommercial-Sha...
3,3.5,no,aldfly,2 (stereo),2013-05-27,both,33,XC135456.mp3,both,Alder Flycatcher,...,135456,https://www.xeno-canto.org/135456,United States,Mike Nelson,Empidonax alnorum_Alder Flycatcher,-82.1106,0-3(s),08:30,Mike Nelson,Creative Commons Attribution-NonCommercial-Sha...
4,4.0,no,aldfly,2 (stereo),2013-05-27,both,36,XC135457.mp3,level,Alder Flycatcher,...,135457,https://www.xeno-canto.org/135457,United States,Mike Nelson,Empidonax alnorum_Alder Flycatcher,-82.1106,0-3(s),08:30,Mike Nelson,Creative Commons Attribution-NonCommercial-Sha...


In [3]:
def train():
    wandb.init()

    data_path = "./input/birdsong-recognition/train_audio/"

    myds = SoundDS(df, data_path)

    # Random split of 80:20 between training and validation
    num_items = len(myds)
    num_train = round(num_items * 0.8)
    num_val = num_items - num_train
    train_ds, val_ds = random_split(myds, [num_train, num_val])

    # Create training and validation data loaders
    train_dl = torch.utils.data.DataLoader(train_ds,
                                           batch_size=wandb.config.batch_size_train,
                                           shuffle=True)
    val_dl = torch.utils.data.DataLoader(val_ds,
                                         batch_size=wandb.config.batch_size_val,
                                         shuffle=False)

    # Create the model and put it on the GPU if available
    model = AudioClassifier()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    # Check that it is on Cuda
    # var = next(model.parameters()).device

    num_epochs = wandb.config.epochs
    training(model, train_dl, num_epochs, device)

    # Run inference on trained model with the validation set
    inference(model, val_dl, device)
    
    # Save the model in the exchangeable ONNX format
    torch.onnx.export(model, torch.randn(1, 1, 128, 201), "model.onnx")
    
    wandb.save("model.onnx")

In [4]:
sweep_config = {
    "name": "Baseline Sweep",
    "method": "bayes",
    "metric": {"goal": "maximize", "name": "val_acc"},
    "parameters": {
        "epochs": {"min": 7, "max": 20},
        "learning_rate": {"min": 0, "max": 0.1, "distribution": "log_uniform"},
        
        "batch_size_train": {"values": [32]},
        "batch_size_val": {"values": [32]},
        "anneal_strategy": {"values": ["linear"]},
    }
}
sweep_id = wandb.sweep(sweep=sweep_config, project="Baseline", entity="swiss-birder")

wandb.agent(sweep_id, function=train)

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. learning_rate uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: p2kdwd1j
Sweep URL: https://wandb.ai/swiss-birder/Baseline/sweeps/p2kdwd1j


wandb: Agent Starting Run: gvhuhy37 with config:
wandb: 	anneal_strategy: linear
wandb: 	batch_size_train: 16
wandb: 	batch_size_val: 16
wandb: 	epochs: 18
wandb: 	learning_rate: 1.0747457388650192
wandb: Currently logged in as: fender-jakob (swiss-birder). Use `wandb login --relogin` to force relogin


Finished Training
Accuracy: 0.62, Total items: 40


Accuracy,▃▆▅▇▁▃▇▄▇▆▅▇▇▄▇▆█▆
Loss,▁▁▂▇█▇▃▅▄▂▁▁▁▁▁▁▁▁
total_prediction,▁
val_acc,▁
Accuracy,0.54375
Loss,0.70324
total_prediction,40
val_acc,0.625


...

Accuracy,▁▄██▆▅▆
Loss,▁▁██▇▄▄
total_prediction,▁
val_acc,▁
Accuracy,0.5
Loss,5.58548
total_prediction,40
val_acc,0.5


Run 21xxutbr errored:
Traceback (most recent call last):
  File "C:\code\dspro2\.venv\lib\site-packages\wandb\agents\pyagent.py", line 308, in _run_job
    self._function()
  File "C:\Users\fende\AppData\Local\Temp\ipykernel_20384\1851611840.py", line 36, in train
    torch.onnx.export(model, torch.randn(1, 1, 128, 201), "model.onnx")
  File "C:\code\dspro2\.venv\lib\site-packages\torch\onnx\utils.py", line 516, in export
    _export(
  File "C:\code\dspro2\.venv\lib\site-packages\torch\onnx\utils.py", line 1613, in _export
    graph, params_dict, torch_out = _model_to_graph(
  File "C:\code\dspro2\.venv\lib\site-packages\torch\onnx\utils.py", line 1135, in _model_to_graph
    graph, params, torch_out, module = _create_jit_graph(model, args)
  File "C:\code\dspro2\.venv\lib\site-packages\torch\onnx\utils.py", line 1011, in _create_jit_graph
    graph, torch_out = _trace_and_get_graph_from_model(model, args)
  File "C:\code\dspro2\.venv\lib\site-packages\torch\onnx\utils.py", line 915, 

Finished Training
Accuracy: 0.55, Total items: 40


Accuracy,▁▇▅▆▇▆▆█▇
Loss,▁▂▃█▃▂▂▁▁
total_prediction,▁
val_acc,▁
Accuracy,0.5375
Loss,2.35393
total_prediction,40
val_acc,0.55


Run s8jsyr19 errored:
Traceback (most recent call last):
  File "C:\code\dspro2\.venv\lib\site-packages\wandb\agents\pyagent.py", line 308, in _run_job
    self._function()
  File "C:\Users\fende\AppData\Local\Temp\ipykernel_20384\1851611840.py", line 36, in train
    torch.onnx.export(model, torch.randn(1, 1, 128, 201), "model.onnx")
  File "C:\code\dspro2\.venv\lib\site-packages\torch\onnx\utils.py", line 516, in export
    _export(
  File "C:\code\dspro2\.venv\lib\site-packages\torch\onnx\utils.py", line 1613, in _export
    graph, params_dict, torch_out = _model_to_graph(
  File "C:\code\dspro2\.venv\lib\site-packages\torch\onnx\utils.py", line 1135, in _model_to_graph
    graph, params, torch_out, module = _create_jit_graph(model, args)
  File "C:\code\dspro2\.venv\lib\site-packages\torch\onnx\utils.py", line 1011, in _create_jit_graph
    graph, torch_out = _trace_and_get_graph_from_model(model, args)
  File "C:\code\dspro2\.venv\lib\site-packages\torch\onnx\utils.py", line 915, 